In [ ]:
%pip install auto-sklearn

     |████████████████████████████████| 6.4 MB 25.8 MB/s 
     |████████████████████████████████| 38.1 MB 1.3 MB/s 
     |████████████████████████████████| 22.3 MB 1.2 MB/s 
     |████████████████████████████████| 1.1 MB 63.9 MB/s 
     |████████████████████████████████| 837 kB 62.1 MB/s 
     |████████████████████████████████| 4.3 MB 42.2 MB/s 
     |████████████████████████████████| 4.0 MB 52.4 MB/s 
     |████████████████████████████████| 167 kB 49.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 136 kB 72.7 MB/s 
     |████████████████████████████████| 596 kB 63.3 MB/s 
     |████████████████████████████████| 45 kB 4.2 MB/s 
     |████████████████████████████████| 260 kB 55.1 MB/s 
  Created wheel for auto-sklearn: filename=auto_sklearn-0.14.6-py3-none-any.whl size=6602835 sha256=67ca8617de0c933cefb12b8cee591e557b199eeb01a828bb38d446fe05282411
  Stored in dir

In [ ]:
import sklearn.datasets
import sklearn.model_selection
import autosklearn.classification
import autosklearn.metrics
import pandas as pd

In [ ]:
def dataSetup(name):
  X,y = sklearn.datasets.fetch_openml(data_id = name, as_frame=True, return_X_y=True)
  train = pd.concat([X, y], axis=1, join='inner')
  train.dropna()
  train = train.apply(lambda x: pd.factorize(x)[0])
  X,y = train.iloc[:,:-1], train.iloc[:, -1]
  # X = X.apply(lambda x: pd.factorize(x)[0])
  X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y,random_state=42)
  return (X_train, X_test, y_train, y_test)

In [ ]:
X_train, X_test, y_train, y_test = dataSetup(1596)
X_train.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40
442216,212,218,25,18,11,2127,30,37,226,1174,...,0,0,0,0,0,0,0,0,0,0
20198,61,4,1,190,7,4264,1,3,1,491,...,0,0,0,0,0,0,0,0,0,0
76896,274,15,8,12,31,2893,49,25,44,5123,...,0,0,0,0,0,0,0,0,0,0
349647,1430,38,7,12,100,2348,3,3,82,1996,...,0,1,0,0,0,0,0,0,0,0
440763,991,118,12,124,91,2209,44,54,32,1045,...,0,1,0,0,0,0,0,0,0,0


In [ ]:
estimator_askl = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=3600,seed=42,resampling_strategy='cv',)
estimator_askl.fit(X_train, y_train, dataset_name='car')
prediction = estimator_askl.predict(X_test)

In [ ]:
(prediction == y_test).sum()/len(prediction)

0.5076659346106449

In [ ]:
print(estimator_askl.sprint_statistics())

auto-sklearn results:
  Dataset name: car
  Metric: accuracy
  Best validation score: 0.494126
  Number of target algorithm runs: 17
  Number of successful target algorithm runs: 1
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 9
  Number of target algorithms that exceeded the memory limit: 7



In [ ]:
from pprint import pprint
pprint(estimator_askl.show_models(), indent=4)

{   6: {   'cost': 0.5058736595228096,
           'ensemble_weight': 1.0,
           'estimators': [   {   'balancing': Balancing(random_state=42),
                                 'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice object at 0x7f53facdbb90>,
                                 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7f53faccff90>,
                                 'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7f53facea1d0>,
                                 'sklearn_classifier': PassiveAggressiveClassifier(C=0.14268277711454813, max_iter=128,
                            random_state=42, tol=0.0002600768160857831,
                            warm_start=True)},
                             {   'balancing': Balancing(random_state=42),
                                 'classifier': <autosklearn.pipeline.components

In [ ]:
for run_key in estimator_askl.automl_.runhistory_.data:
    print('#########')
    print(run_key)
    print(estimator_askl.automl_.runhistory_.data[run_key])

#########
RunKey(config_id=1, instance_id='{"task_id": "car"}', seed=0, budget=0.0)
RunValue(cost=1.0, time=38.51653337478638, status=<StatusType.MEMOUT: 5>, starttime=1651517850.206973, endtime=1651517889.7753909, additional_info={'error': 'Memout (used more than 3072 MB).', 'configuration_origin': 'Initial design'})
#########
RunKey(config_id=2, instance_id='{"task_id": "car"}', seed=0, budget=0.0)
RunValue(cost=1.0, time=360.02615785598755, status=<StatusType.TIMEOUT: 2>, starttime=1651517890.2352374, endtime=1651518251.2933915, additional_info={'error': 'Timeout', 'configuration_origin': 'Initial design'})
#########
RunKey(config_id=3, instance_id='{"task_id": "car"}', seed=0, budget=0.0)
RunValue(cost=1.0, time=40.699110984802246, status=<StatusType.MEMOUT: 5>, starttime=1651518251.5949097, endtime=1651518293.3483236, additional_info={'error': 'Memout (used more than 3072 MB).', 'configuration_origin': 'Initial design'})
#########
RunKey(config_id=4, instance_id='{"task_id": "car"

In [ ]:
print("Accuracy score", sklearn.metrics.accuracy_score(y_test, prediction))

Accuracy score 0.5076659346106449
